In [88]:
import pandas as pd
import os

#libreria para el modelo
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## Carga de Dataframe

In [89]:
df1_redu = pd.read_csv(r'dataframe\df1_redu.csv')

In [87]:
df1_redu.shape

(2250, 7)

Se uso un 20% del dataframe original porque al momento de hacer el render se sobrepasaba la capasidad de ram que ofrece la cuenta gratuita. Se paso de 22496 filas a 2250

In [81]:
#  df1_redu = df1.sample(frac=0.1)
#  df1_redu.reset_index(drop=True, inplace=True)
#  df1_redu[df1_redu['id']==670290]

,genres,title,tags,id,developer,Añio,price
1545,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...",670290,Poolians.com,2017,0.0


In [83]:
#df1_redu.to_csv(os.path.join('dataframe','df1_redu.csv'), index=False)

## Crea la matriz de Cosine Similarity

In [91]:
#Defina un objeto vectorizador TF-IDF. Elimine todas las palabras vacías en inglés, como 'the', 'a' 
tfidf = TfidfVectorizer (stop_words="english")

#Construya la matriz TF-IDF requerida ajustando y transformando Los datos
tfidf_matrix = tfidf.fit_transform(df1_redu['genres'])

#Salida de la forma de tfidf_matrix
#print(tfidf_matrix.shape)
#Matriz de 22496 filas por 31 columnas

#Dado que hemos utilizado el vectorizador TF-IDF, el cálculo del producto escalar nos dará directamente 
#la puntuación de similitud del coseno. Por lo tanto, usaremos linear_kernel () de sklearn en lugar de 
#cosine_similarities() ya que es más rápido.

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#necesitamos un mecanismo para identificar el índice de una película en nuestro DataFrame de metadatos, dado su título.

#Construya un mapa inverso de índices y títulos de películas.
indices = pd.Series (df1_redu.index, index=df1_redu['id']).drop_duplicates()



## Para Guardar la matriz cosine_sim
 Pero pesa demasiado, mas de 500 MB para el dataframe completo, no es una buena opcion 

In [ ]:
#Se guarda cosine_sim
#(pd.DataFrame(cosine_sim)).to_parquet(os.path.join('dataframe','cosine_sim.parquet'))
#Para abrir el archivo
#cosine_sim = np.array(pd.read_parquet(r'dataframe\cosine_sim.parquet.gzip'))


## Funcion de Sistema de Recomendacion

In [92]:
#Definamos una función que tome el título de una película como entrada y genere una 
#lista de las 10 películas más similares.

def recomendacion_juego (juego_id, cosine_sim=cosine_sim):

    idx = indices[juego_id]

# Obtenga las puntuaciones de similitud por pares de todas las películas con esa película
    sim_scores = list(enumerate (cosine_sim[idx]))

# Ordene las películas según las puntuaciones de similitud 
    sim_scores = sorted (sim_scores, key=lambda x: x[1], reverse=True)

# Obtén las puntuaciones de las 10 películas más similares 
    sim_scores = sim_scores [1:11]

# Obtenga Los índices de películas

    movie_indices = [i[0] for i in sim_scores]

# Devuelve el top 10 de películas más similares 
    lista = list(df1_redu['id'].iloc[movie_indices])
    lista = lista[0:10]
    

    #Para sacar juegos sin repeticion en los recomendados
    a=None
    lista_juegos=[]
    for i,l in enumerate(lista):
        if not(l in lista_juegos) and (l != id):
           lista_juegos.append(lista[i])
    #del lista_juegos[2]
    
    
    return {
          'Juegos recomendados a partir del juego juego_id' : str(juego_id), 
          'No 1': str(df1_redu[df1_redu['id']==lista_juegos[0]].iloc[0][1]),
          'No 2': str(df1_redu[df1_redu['id']==lista_juegos[1]].iloc[0][1]),
          'No 3': str(df1_redu[df1_redu['id']==lista_juegos[2]].iloc[0][1]),
          'No 4': str(df1_redu[df1_redu['id']==lista_juegos[3]].iloc[0][1]),
          'No 5': str(df1_redu[df1_redu['id']==lista_juegos[4]].iloc[0][1])
          }

In [93]:
recomendacion_juego (670290)



{'Juegos recomendados a partir del juego juego_id': '670290',
 'No 1': 'Zaccaria Pinball - Lucky Fruit Table',
 'No 2': 'Zaccaria Pinball - Moon Flight Table',
 'No 3': 'Zaccaria Pinball - Soccer Kings Table',
 'No 4': 'Pixel Puzzles Ultimate - Puzzle Pack: Desserts',
 'No 5': 'Pixel Puzzles Ultimate - Puzzle Pack: Rio'}

Salida producida entremando el modelo con todos los datos

 ```
{'Juegos recomendados a partir del juego juego_id': '670290',
 'No 1': 'Snooker-online multiplayer snooker game!',
 'No 2': "Malzbie's Pinball Collection",
 'No 3': 'Zaccaria Pinball - Blackbelt Table',
 'No 4': 'Zaccaria Pinball - Bronze Membership',
 'No 5': 'Zaccaria Pinball - Cine Star Table'}
```